In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Hyperparameter Tuning for Deep Learning Models (running cca 5min without GPU)

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called hyperparameter tuning or hypertuning.

URL: https://keras.io/guides/keras_tuner/getting_started/

### Setup

In [ ]:
# !pip install tensorflow keras-tuner -q

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt

In [ ]:
import keras_tuner as kt

In [ ]:
def plot_graphs(history):
    plt.subplots(figsize=(15, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    
    plt.show()

### Load MNIST

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0


### Baseline Architecture

In [ ]:
from keras import layers

def baseline_model(units, activation, dropout, lr):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(units=units, activation=activation))
    if dropout:
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    return model


### Define Hyperparameter Space

In [ ]:
def build_model(hp):
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    dropout = hp.Boolean("dropout")
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # call existing model-building code with the hyperparameter values.
    model = baseline_model(
        units=units, activation=activation, dropout=dropout, lr=lr
    )
    return model

build_model(kt.HyperParameters())

### Instantiate the tuner and perform hypertuning

The Keras Tuner currently supports four tuners:
- `RandomSearch`
- `Hyperband`
- `BayesianOptimization`
- `Sklearn`

In [ ]:
# Define tuner and parameters
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Define callbacks
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search_space_summary()

In [ ]:
# Run Tuner
tuner.search(img_train, label_train, epochs=10, validation_split=0.2, callbacks=[stop_early])

### Explore Hyperparameters

In [ ]:
import pandas as pd

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

pd.DataFrame([_.values for _ in tuner.get_best_hyperparameters(5)])

In [ ]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()

In [ ]:
tuner.results_summary()

### Train the Model

#### Find the optimal number of epochs to train the model with the hyperparameters obtained from search

In [ ]:
model = tuner.hypermodel.build(best_hps)
best_model_history = model.fit(img_train, label_train, epochs=10, validation_split=0.2)


val_acc_per_epoch = best_model_history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print(f'Best epoch: {best_epoch}')

In [ ]:
plot_graphs(best_model_history)

#### Retrain the model with optimal number of epochs

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
history = hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

In [ ]:
# Plot graphs
plot_graphs(best_model_history)

#### Evaluate the Model

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print(f'[test loss, test accuracy]: {eval_result}')

The `my_dir/intro_to_kt` directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional overwrite=True argument while instantiating the tuner.